# Libraries

In [13]:
import numpy as np
import pandas as pd

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

import pickle

from sklearn.linear_model import LogisticRegression

# Data Preparation


In [4]:
#import Scalar
scaler = pickle.load(open('scaler.pickle','rb'))

#import PCA
pca = pickle.load(open('pca.pickle','rb'))
#import k-means
#kmeans_pca = pickle.load(open('kmeans_pca.pickle','rb'))

In [7]:
#import data
df_purchase = pd.read_csv('purchase data.csv')
df_purchase.head()

,ID,Day,Incidence,Brand,Quantity,Last_Inc_Brand,Last_Inc_Quantity,Price_1,Price_2,Price_3,...,Promotion_3,Promotion_4,Promotion_5,Sex,Marital status,Age,Education,Income,Occupation,Settlement size
0,200000001,1,0,0,0,0,0,1.59,1.87,2.01,...,0,0,0,0,0,47,1,110866,1,0
1,200000001,11,0,0,0,0,0,1.51,1.89,1.99,...,0,0,0,0,0,47,1,110866,1,0
2,200000001,12,0,0,0,0,0,1.51,1.89,1.99,...,0,0,0,0,0,47,1,110866,1,0
3,200000001,16,0,0,0,0,0,1.52,1.89,1.98,...,0,0,0,0,0,47,1,110866,1,0
4,200000001,18,0,0,0,0,0,1.52,1.89,1.99,...,0,0,0,0,0,47,1,110866,1,0


In [8]:
# Standardization
features = df_purchase[['Sex','Marital status','Age','Education','Income','Occupation','Settlement size']]
df_purchase_segm_std = scaler.transform(features)

In [9]:
#apply pca
df_purchase_segm_pca = pca.transform(df_purchase_segm_std)

In [10]:
#segment data 
purchase_segm_kmeans_pca = kmeans_pca.predict(df_purchase_segm_pca)

In [12]:
#create a copy of dataframe
df_purchase_predictors = df_purchase.copy()
#add segment label
df_purchase_predictors['Segment'] = purchase_segm_kmeans_pca
segment_dummies = pd.get_dummies(purchase_segm_kmeans_pca,prefix='Segment',prefix_sep='_')
df_purchase_predictors = pd.concat([df_purchase_predictors,segment_dummies],axis = 1)

#new name here
df_pa = df_purchase_predictors

# Purchase Probability Model

In [ ]:
#chose dependeny and independent var
#dep var = whether purchase can take place or not it is in incidence var
#purchase prob is influenced by price

#we want to know whwther the brand is purchased by its price it occur or not
#price indicator = min/max , mean/medium price, 

In [19]:
Y = df_pa['Incidence']

In [17]:
X = pd.DataFrame()
X['Mean_price'] = (df_pa['Price_1']+
                   df_pa['Price_2']+
                   df_pa['Price_3']+
                   df_pa['Price_4']+
                   df_pa['Price_5']
                  ) / 5

In [21]:
model_purchase = LogisticRegression(solver = 'sag')
model_purchase.fit(X,Y)

LogisticRegression(solver='sag')

In [22]:
#this command estimate prob of purchase model and store in model incidence object
model_purchase.coef_
#coeff of indep variable which is price 
#-2.3 shows decrease inp price will lead to increase in incidnce
#change in purchase beh is shown by log regression coeff

array([[-2.34827817]])

# Price Elasticity of purchase Probability

In [23]:
# we will analyse the range of prices
df_pa[['Price_1','Price_2','Price_3','Price_4','Price_5']].describe()

,Price_1,Price_2,Price_3,Price_4,Price_5
count,58693.000000,58693.000000,58693.000000,58693.000000,58693.000000
mean,1.392074,1.780999,2.006789,2.159945,2.654798
std,0.091139,0.170868,0.046867,0.089825,0.098272
min,1.100000,1.260000,1.870000,1.760000,2.110000
25%,1.340000,1.580000,1.970000,2.120000,2.630000
50%,1.390000,1.880000,2.010000,2.170000,2.670000
75%,1.470000,1.890000,2.060000,2.240000,2.700000
max,1.590000,1.900000,2.140000,2.260000,2.800000
